Exploring a dataset with information about patients. 
Consider next 8 features to predict the mortality rate:
1. Age
2. ОССН KiLLip
3. HBR (b)
4. Systolic AP(b)
5. Creatine in blood
6. EF Percentage
7. White blood cells count
8. Glucose

1. Import required libraries and load the dataset

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

import shap

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import display

# Load dataset
# data = pd.read_excel('./import/DataSet_V47.xlsx')
data = pd.read_csv('./import/subset13columns.csv')

1.1. Select features

In [54]:
columns_in_data = ['Age', 
           'ОССН KiLLip', 
           'ЧСС (b)', 
           'Систолическое АД(b)', 
           'Креатинин', 
           'EF%',
           'Лейкоциты(b)',
           'Глюкоза(a)',
           'начало операции',
           'Дата операции в БД',
           'дата выписки',
           'дата смерти',
           'Исход заболевания']

display(data.describe())
display(data.isna().sum())

columns_with_na = ['Age',
                   'ЧСС (b)', 
                   'Систолическое АД(b)', 
                   'Креатинин',
                   'EF%', 
                   'Лейкоциты(b)',  
                   'Глюкоза(a)']
filtered_subset_tests_no_nan = data.dropna(subset=columns_with_na)
display(filtered_subset_tests_no_nan.describe())
display(filtered_subset_tests_no_nan.isna().sum())



# X = data.drop('дата смерти')
# y = data['дата смерти']

,Unnamed: 0,Age,ОССН KiLLip,ЧСС (b),Систолическое АД(b),Креатинин,EF%,Лейкоциты(b),Глюкоза(a)
count,12538.000000,12510.000000,11622.000000,11870.000000,11783.000000,11441.000000,9314.000000,7472.000000,9195.000000
mean,6268.500000,62.657714,1.378506,71.996504,134.418909,106.250302,56.464784,10.122987,6.550492
std,3619.553172,10.925874,0.963044,19.794550,122.307101,53.376197,9.043406,4.001209,3.040381
min,0.000000,17.000000,0.000000,-1.000000,-1.000000,0.000000,10.000000,0.000000,0.000000
25%,3134.250000,56.000000,1.000000,64.000000,120.000000,83.000000,50.000000,7.300000,5.080000
50%,6268.500000,63.000000,1.000000,70.000000,130.000000,98.210000,58.000000,9.400000,5.790000
75%,9402.750000,70.000000,2.000000,80.000000,150.000000,116.000000,63.000000,12.100000,6.995000
max,12537.000000,95.000000,4.000000,900.000000,13000.000000,1354.000000,97.000000,43.100000,116.530000


Unnamed: 0                 0
Age                       28
ОССН KiLLip              916
ЧСС (b)                  668
Систолическое АД(b)      755
Креатинин               1097
EF%                     3224
Лейкоциты(b)            5066
Глюкоза(a)              3343
начало операции         4566
Дата операции в БД      2329
дата выписки            3487
дата смерти            12053
Исход заболевания        926
dtype: int64

,Unnamed: 0,Age,ОССН KiLLip,ЧСС (b),Систолическое АД(b),Креатинин,EF%,Лейкоциты(b),Глюкоза(a)
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,6247.245167,62.735167,1.498833,72.854583,136.029167,106.798890,56.171667,10.349603,6.514468
std,3601.724044,10.498273,0.896920,16.328817,168.079058,53.768507,8.958120,3.946659,3.178992
min,2.000000,21.000000,0.000000,-1.000000,-1.000000,0.000000,18.000000,0.000000,0.000000
25%,3107.500000,56.000000,1.000000,65.000000,120.000000,83.700000,50.000000,7.515000,5.060000
50%,6229.000000,63.000000,1.000000,70.000000,130.000000,99.135000,57.000000,9.610000,5.770000
75%,9376.250000,70.000000,2.000000,80.000000,150.000000,117.440000,62.000000,12.400000,6.930000
max,12523.000000,94.000000,4.000000,170.000000,13000.000000,1354.000000,97.000000,41.900000,116.530000


Unnamed: 0                0
Age                       0
ОССН KiLLip               0
ЧСС (b)                   0
Систолическое АД(b)       0
Креатинин                 0
EF%                       0
Лейкоциты(b)              0
Глюкоза(a)                0
начало операции        2161
Дата операции в БД        0
дата выписки           2166
дата смерти            5875
Исход заболевания         1
dtype: int64

1.2. Explore how NaN values in 'дата выписки', 'дата смерти' are related to each other and to 'Исход заболевания'. 
Count records where all two are NaN:

In [82]:

begin_end_op_and_death_dates_nan = filtered_subset_tests_no_nan.loc[filtered_subset_tests_no_nan['дата выписки'].isna()
                           &filtered_subset_tests_no_nan['дата смерти'].isna()]
display(begin_end_op_and_death_dates_nan.groupby('Исход заболевания').size())

Исход заболевания
без перемен                                     4
выписан с улучшением                            1
неоконченный случай с улучшением                1
переведен в другой стационар с улучшением       6
с выздоровлением                                1
с улучшением                                 2101
самоуход                                        1
умер                                            9
dtype: int64

There are 9 death cases without any dates. 

1.3. Examine dates and patients' conditons

In [91]:
display(filtered_subset_tests_no_nan['Дата операции в БД'].dtype)
display(filtered_subset_tests_no_nan['дата выписки'].dtype) 
display(filtered_subset_tests_no_nan['дата смерти'].dtype)

dtype('<M8[ns]')

dtype('O')

dtype('O')

In [158]:
filtered_subset_tests_no_nan_copy = filtered_subset_tests_no_nan.copy()
for col_name in ['Дата операции в БД', 'дата выписки', 'дата смерти']:
    new_col_name = f"{col_name}_parsed"
    filtered_subset_tests_no_nan_copy[new_col_name] = pd.to_datetime(filtered_subset_tests_no_nan_copy[col_name], errors='coerce')

There are some invalid date values.
Looking at rows where parsed values are NaN and original values are not NaN

In [118]:
mask = (
    (filtered_subset_tests_no_nan_copy['дата выписки_parsed'].isna() & filtered_subset_tests_no_nan_copy['дата выписки'].notna())
    |(filtered_subset_tests_no_nan_copy['дата смерти_parsed'].isna() & filtered_subset_tests_no_nan_copy['дата смерти'].notna())
)
filtered_subset_tests_no_nan_copy.loc[mask, ['дата выписки', 'дата смерти', 'Исход заболевания']]

,дата выписки,дата смерти,Исход заболевания
659,2020-01-20 20:00:00,Да,умер
1694,2013-02-08 09:55:00,Да,с улучшением
2714,2017-06-11 08:51:00,Да,умер
4166,2018-06-23 10:05:00,Да,умер
7542,2019-01-22 23:00:00,Да,умер
9959,: 09.01.2009 эффективность лечения,Да,с улучшением
10101,2017-01-23 12:25:00,Да,умер
10329,2020-02-05 11:30:00,Да,умер
11788,2016-04-19 08:10:00,Да,с улучшением


These rows are umbiguous, will drop them.

In [119]:
rows_to_keep = ~mask
filtered_data = filtered_subset_tests_no_nan_copy[rows_to_keep]

Now checking NaN 'дата смерти' and 'Исход заболевания'.

In [161]:
query_expression = "(not `дата смерти`.isnull() and `Исход заболевания` != 'умер')"

filtered_data.query(query_expression).head(50)

,Unnamed: 0,Age,ОССН KiLLip,ЧСС (b),Систолическое АД(b),Креатинин,EF%,Лейкоциты(b),Глюкоза(a),начало операции,Дата операции в БД,дата выписки,дата смерти,Исход заболевания,Дата операции в БД_parsed,дата выписки_parsed,дата смерти_parsed
3989,3989,76.0,1.0,92.0,120.0,123.72,62.0,9.3,6.32,2016-12-01 18:00:00,2016-10-14,2016-12-01 21:20:00,2016-12-01 21:20:00,с улучшением,2016-10-14,2016-12-01 21:20:00,2016-12-01 21:20:00
9761,9761,61.0,3.0,72.0,130.0,108.00,65.0,12.9,6.37,2016-12-29 15:00:00,2016-12-29,2017-01-02 18:51:00,2017-01-02 00:00:00,самоуход,2016-12-29,2017-01-02 18:51:00,2017-01-02 00:00:00


It's ok, it seems for these two rows the data in the column "Дата смерти" was put there by a mistake.

Keeping only death cases within 55 days from the operation date.
- If there no information about "Дата смерти", but "Исход заболевания" is "умер", we will consider "Дата выписки" as the day of death.

In [170]:
mask = (
    (filtered_data['дата смерти_parsed'].notna()) & 
                (filtered_data['дата смерти_parsed'] - filtered_data['Дата операции в БД_parsed'] > pd.Timedelta('55 days'))
    | ((filtered_data['дата смерти_parsed'].isna()) & 
                (filtered_data['Исход заболевания'] == 'умер') & 
                (filtered_data['дата выписки_parsed'].notna()) & 
                (filtered_data['дата выписки_parsed'] - filtered_data['Дата операции в БД_parsed'] > pd.Timedelta('55 days'))
        )
)

rows_to_keep = ~mask

In [171]:
print(f"Rows to drop: {filtered_data[mask].shape[0]}")
proper_date_subset = filtered_data[rows_to_keep]
print(f"Rows left in the dataset: {proper_date_subset.shape[0]}")

Rows to drop: 3
Rows left in the dataset: 5988


2. Split the dataset into train, fit, and test sets

In [186]:
X = proper_date_subset[['Age', 
                        'ОССН KiLLip', 
                        'ЧСС (b)', 
                        'Систолическое АД(b)', 
                        'Креатинин', 
                        'EF%', 
                        'Лейкоциты(b)', 
                        'Глюкоза(a)']]
y = proper_date_subset['Исход заболевания'].apply(lambda x: 1 if x == 'умер' else 0)

In [187]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_fit, X_test, y_fit, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


3. Data preprocessing pipeline

In [ ]:
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor.fit(X_train)

X_train_preprocessed = preprocessor.transform(X_train)
X_fit_preprocessed = preprocessor.transform(X_fit)
X_test_preprocessed = preprocessor.transform(X_test)

4. Linear Regression models (Univariative and Multivariate)

In [ ]:
# Univariative Linear Regression
for feature_name in X.columns:
    lr = LinearRegression()
    lr.fit(X_train_preprocessed[:, [X.columns.get_loc(feature_name)]], y_train)
    
    # Feature weight estimation
    print(f"Feature weight for {feature_name}: {lr.coef_[0]}")

# Multivariate Linear Regression
lr_multi = LinearRegression()
lr_multi.fit(X_train_preprocessed, y_train)

# Feature weight estimation
for feature_name, weight in zip(X.columns, lr_multi.coef_):
    print(f"Feature weight for {feature_name}: {weight}")

# Shapley values for Multivariate Linear Regression
explainer = shap.Explainer(lr_multi, X_train_preprocessed, feature_names=X.columns)
shap_values = explainer(X_fit_preprocessed)
shap.summary_plot(shap_values, X_fit, plot_type='bar')
plt.title('Shapley Values for Multivariate Linear Regression')
plt.show()


5. Gradient Boosting model

In [ ]:
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train_preprocessed, y_train)

# Feature importance via impurity
feature_importances_gb = gb.feature_importances_

# Permutation importance
result_gb = permutation_importance(gb, X_fit_preprocessed, y_fit, n_repeats=10, random_state=42)
perm_importances_gb = result_gb.importances_mean

# Shapley values for Gradient Boosting
explainer = shap.Explainer(gb, X_train_preprocessed, feature_names=X.columns)
shap_values = explainer(X_fit_preprocessed)
shap.summary_plot(shap_values, X_fit, plot_type='bar')
plt.title('Shapley Values for Gradient Boosting')
plt.show()

6. Random Forest model

In [ ]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_preprocessed, y_train)

# Feature importance via impurity
feature_importances_rf = rf.feature_importances_

# Permutation importance
result_rf = permutation_importance(rf, X_fit_preprocessed, y_fit, n_repeats=10, random_state=42)
perm_importances_rf = result_rf.importances_mean

# Shapley values for Random Forest
explainer = shap.Explainer(rf, X_train_preprocessed, feature_names=X.columns)
shap_values = explainer(X_fit_preprocessed)
shap.summary_plot(shap_values, X_fit, plot_type='bar')
plt.title('Shapley Values for Random Forest')
plt.show()

7. Deep Neural Network model

In [ ]:
def create_dnn(input_shape, num_layers, units_per_layer, l1_l2_reg, dropout_rate):
    model = Sequential()
    model.add(Dense(units_per_layer, input_shape=input_shape, activation='relu', kernel_regularizer=L1L2(*l1_l2_reg)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(units_per_layer, activation='relu', kernel_regularizer=L1L2(*l1_l2_reg)))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

def dnn_cross_val_score(X, y, num_layers, units_per_layer, l1_l2_reg, dropout_rate, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kfold.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        preprocessor.fit(X_train)
        X_train_preprocessed = preprocessor.transform(X_train)
        X_val_preprocessed = preprocessor.transform(X_val)

        dnn = create_dnn((X_train_preprocessed.shape[1],), num_layers, units_per_layer, l1_l2_reg, dropout_rate)
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        dnn.fit(X_train_preprocessed, 
                y_train, 
                validation_data=(X_val_preprocessed, y_val), 
                epochs=100, 
                batch_size=32, 
                verbose=0, 
                callbacks=[early_stopping]
                )

        score = dnn.evaluate(X_val_preprocessed, y_val, verbose=0)
        scores.append(score)

    return np.mean(scores)

num_layers = 2
units_per_layer = 64
l1_l2_reg = (1e-5, 1e-5)
dropout_rate = 0.2
mean_score = dnn_cross_val_score(X, y, num_layers, units_per_layer, l1_l2_reg, dropout_rate)
print(f"Mean score for {num_layers} layers, {units_per_layer} units per layer, L1/L2 reg = {l1_l2_reg}, dropout rate = {dropout_rate}: {mean_score}")

In [ ]:
dnn = create_dnn((X_train_preprocessed.shape[1],), num_layers, units_per_layer, l1_l2_reg, dropout_rate)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
dnn.fit(X_train_preprocessed, y_train, validation_data=(X_fit_preprocessed, y_fit), epochs=100, batch_size=32, callbacks=[early_stopping])

# Shapley values for Deep Neural Network
explainer = shap.Explainer(dnn, X_train_preprocessed, feature_names=X.columns)
shap_values = explainer(X_fit_preprocessed)
shap.summary_plot(shap_values, X_fit, plot_type='bar')
plt.title('Shapley Values for Deep Neural Network')
plt.show()

8. Comparison of feature importances

In [ ]:
feature_importance_df = pd.DataFrame({'Feature': X.columns,
                                      'Multivariate Linear Regression': lr_multi.coef_,
                                      'Gradient Boosting (impurity)': feature_importances_gb,
                                      'Gradient Boosting (permutation)': perm_importances_gb,
                                      'Random Forest (impurity)': feature_importances_rf,
                                      'Random Forest (permutation)': perm_importances_rf})

feature_importance_df.set_index('Feature', inplace=True)
feature_importance_df.plot(kind='bar', figsize=(12, 6))
plt.title('Feature Importances for Different Models')
plt.ylabel('Importance')
plt.xticks(rotation=45)
plt.show()